# 散乱強度・距離の補正
- プロファイル関連の特徴量が変化
- 画像特徴量は変化なしなので、今回は生成しない

In [1]:
import os
import time
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit, minimize_scalar
from scipy import signal
from sklearn.metrics import r2_score, mean_squared_error

from FuncLaserScattering import delete_distance, smoothing_apple, skip_distance, correct_intensity, correct_distance
from FuncLaserScattering import fitting, calc_diff, image2feature

In [2]:
%%time
# 新しい解析法で得られたデータで特徴量作成
# 粉質化のみの算出
for condition in ['FirstStorage','SecondStorage']:
    for storage in tqdm(['00','01','02','03','04']):
        for wave in ['633','850']:

            ##### データの読み込み #####
            total_df = pd.read_csv(f'./../data/data_{condition}/data_laser/Profile_new_data/{wave}nm/Profile_{condition}_{storage}_{wave}nm.csv')

            # 距離と輝度（強度）を分割
            distance = total_df.iloc[:,0]
            apple_df = total_df.drop('distance (mm)',axis=1)


            #####解析範囲外の領域を削除#####
            distance_30mm, apple_df_30mm = delete_distance(distance, apple_df)
          
            #####散乱距離の補正#####
            distance_30mm = correct_distance(distance_30mm)
            
            #####散乱強度の補正#####
            apple_df_30mm = correct_intensity(distance_30mm,apple_df_30mm)

            #####プロファイルの平滑化#####
            distance_smooth, apple_smooth = smoothing_apple(distance_30mm, apple_df_30mm)

            #####プロファイルの間引き (1mm間隔)#####
            distance_eq, apple_smooth_eq = skip_distance(distance_smooth, apple_smooth)


            #####変化率の取得#####
            res = calc_diff(distance_eq, data_smoothing=apple_smooth_eq)

            #####CurveFitting#####
            eff_df = pd.DataFrame()

            distance_eq = distance_eq[1:]
            apple_smooth_eq = apple_smooth_eq.iloc[1:,:]

            for i in range(apple_smooth_eq.shape[1]): 

                eff_temp = fitting(distance_eq,apple_smooth_eq.iloc[:,i])

                eff_df = pd.concat([eff_df,eff_temp],axis=0)

            eff_df.index = res.columns

            # 結合の都合上転置する
            eff_df = eff_df.T
            
             #####画像から特徴量を抽出#####
            print('start image analysis!!!')

            image_path = f'./../data/data_{condition}/data_laser/HDRimage/Original_func/{condition}/{wave}nm/week{storage}'
            feature_img = image2feature(image_path)
            feature_img.index = res.columns


            #####全特徴量の結合および保存#####
            feature_df = pd.concat([eff_df, res, feature_img.T],axis=0).T
            
            

            ##### データの保存 #####
            # 平滑化後のプロファイルおよび特徴量を保存
            apple_smooth_eq.to_csv(f'./../data/data_{condition}/data_laser/smoothing/smoothing_CB_CorrectBoth/{wave}nm/Smooth_{storage}.csv')
            feature_df.to_csv(f'./../data/data_{condition}/data_laser/feature/feature_CB_HDR/feature_all/{wave}nm/feature_{storage}.csv')

            # 分析用にindexを分ける
            index_All = pd.Series(feature_df.index)
            index_Shake = index_All[index_All.str.endswith(('_2','_4'))].tolist()

            # texture推定用に加工した特徴量dataframeを保存
            # XX_X_1, XX_X_3となっているサンプルはテクスチャー推定用に保存
            temp = feature_df.drop(index_Shake,axis=0)
            temp.to_csv(f'./../data/data_{condition}/data_laser/feature/feature_CB_HDR/feature_texture/{wave}nm/feature_{storage}.csv')

            # XX_X_2, XX_X_4となっているサンプルは用意果肉ディスク振とう用に保存
            temp = feature_df.loc[index_Shake,:]
            temp.to_csv(f'./../data/data_{condition}/data_laser/feature/feature_CB_HDR/feature_shake/{wave}nm/feature_{storage}.csv')

  0%|          | 0/5 [00:00<?, ?it/s]

start image analysis!!!


  0%|          | 0/5 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [3]:
# 貯蔵期間ごとの特徴量を１つにまとめる #####

for condition in ['FirstStorage','SecondStorage']:
    for folder in ['feature_all','feature_texture','feature_shake']:
        for wavelen in ["633nm","850nm"]:

            allfile = os.listdir(f'./../data/data_{condition}/data_laser/feature/feature_CB_HDR/{folder}/{wavelen}')
            csv_list = [csvfile for csvfile in allfile if 'feature' in csvfile]
            csv_list = [csvfile for csvfile in csv_list if 'all' not in csvfile]
            csv_list.sort()
            # csv_list.pop(0)
            print(f'{wavelen}における結合するcsvファイル',csv_list)

            # 特徴量を結合
            profile_all = pd.DataFrame()
            for csv_path in csv_list:
                df_temp = pd.read_csv(f'./../data/data_{condition}/data_laser/feature/feature_CB_HDR/{folder}/{wavelen}/{csv_path}',
                                      header=0,
                                      index_col=0)
                profile_all = pd.concat([profile_all, df_temp],axis=0)

            # 特徴量を保存
            feature_all = profile_all
            feature_all.to_csv(f'./../data/data_{condition}/data_laser/feature/feature_CB_HDR/{folder}/{wavelen}/feature_all.csv')

print()
print()
##### 貯蔵期間ごとのprofileを１つにまとめる #####
for condition in ['FirstStorage','SecondStorage']:
    for wavelen in ["633nm","850nm"]:

        allfile = os.listdir(f'./../data/data_{condition}/data_laser/smoothing/smoothing_CB_CorrectBoth/{wavelen}')
        csv_list = [csvfile for csvfile in allfile if 'Smooth' in csvfile]
        csv_list = [csvfile for csvfile in csv_list if 'all' not in csvfile]
        csv_list.sort()
        # csv_list.pop(0)
        print(f'{wavelen}における結合するcsvファイル',csv_list)

        # 特徴量を結合
        profile_all = pd.DataFrame()
        for csv_path in csv_list:
            df_temp = pd.read_csv(f'./../data/data_{condition}/data_laser/smoothing/smoothing_CB_CorrectBoth/{wavelen}/{csv_path}',
                                  header=0,
                                  index_col=0)
            profile_all = pd.concat([profile_all, df_temp],axis=1)

        # 特徴量を保存
        profile_all.to_csv(f'./../data/data_{condition}/data_laser/smoothing/smoothing_CB_CorrectBoth/{wavelen}/Smooth_all.csv')


633nmにおける結合するcsvファイル ['feature_00.csv', 'feature_01.csv', 'feature_02.csv', 'feature_03.csv', 'feature_04.csv']
850nmにおける結合するcsvファイル ['feature_00.csv', 'feature_01.csv', 'feature_02.csv', 'feature_03.csv', 'feature_04.csv']
633nmにおける結合するcsvファイル ['feature_00.csv', 'feature_01.csv', 'feature_02.csv', 'feature_03.csv', 'feature_04.csv']
850nmにおける結合するcsvファイル ['feature_00.csv', 'feature_01.csv', 'feature_02.csv', 'feature_03.csv', 'feature_04.csv']
633nmにおける結合するcsvファイル ['feature_00.csv', 'feature_01.csv', 'feature_02.csv', 'feature_03.csv', 'feature_04.csv']
850nmにおける結合するcsvファイル ['feature_00.csv', 'feature_01.csv', 'feature_02.csv', 'feature_03.csv', 'feature_04.csv']
633nmにおける結合するcsvファイル ['feature_00.csv', 'feature_01.csv', 'feature_02.csv', 'feature_03.csv', 'feature_04.csv']
850nmにおける結合するcsvファイル ['feature_00.csv', 'feature_01.csv', 'feature_02.csv', 'feature_03.csv', 'feature_04.csv']
633nmにおける結合するcsvファイル ['feature_00.csv', 'feature_01.csv', 'feature_02.csv', 'feature_03.csv', 'feature_0